# ACT-R Simon Task

This script is to play with the ACT-R Simon Model created by A.S.(2017)

The Simon Task demo is here: https://www.psytoolkit.org/experiment-library/experiment_simon.html 

<img src="https://ars.els-cdn.com/content/image/1-s2.0-S0010027717300598-gr6.jpg" width="400"/>

# Updated ACT-R Simon Model with Motivation Component

This updated model (simon-motivation-model) implements a simple motivation component in GOAL buffer 


<img src="https://docs.google.com/drawings/d/e/2PACX-1vS_YKK6E75H-XYmQMRBH1xLt7vjmDSMrB0Ykgw0AnppxTx2KwN5OiReoi77Hr5xkdfKc8kVmMDgrYuQ/pub?w=1359&h=1561" width="400"/>

## Motivation Parameter in count unit

In this model (**simon-motivation-model1**), motivation parameter refers to "the maximum times(counts) of retrieval" before giving a response. (Motivation parameter has to be an positive integer, default value is 1). For instance, 

- motivation parameter = 1, it means that the model would never check the rule. 
- motivation parameter = 2, it means the model only have *ONE* chance of checking the rule. As a problem is detected, the model would redo once. But no matter what rule being retrieved, it will respond with the available answer. 
- motivation paramter = 10, it means that the model would go back *AT MOST 9* times before giving a response. However, if the retrieved rule passes the check (check-pass) fires, there is no need to redo the retrieve, the model would proceed with the correct answer. 


## Motivation Parameter in time unit


In this model(**simon-motivation-model2**), motivation parameter is like a mental clock, refering to "the maximum time (in sec) it would spend before providing response. Now, motivation parameter is in second unit, so it should be a float number. For instance,

- motivation paramter = 1.5, it means that when model's mental clock exceeds motivation clock (> 1.5s), it would stop checking the rule anymore, but if the mental clock is within the motivation clock(< 1.5s), it would continue retrieving and checking the rule. 

## Motivation Parameter + Reward delivery


In this model(**simon-motivation-model3**), similar to model3, motivation parameter is like a mental clock, refering to "the maximum time (in sec) it would spend before providing response. However, the reward delivery mechanisms is different. In this model, how much reward is provided depends on how long the model attempts to retrieve. If motivation parameter is high enought, when production `check-pass` fires, it delivers a reward which is equal to the duration of time the model takes since fixation cross appearing on the screen. If motivation parameter is low, the model will not check the answer (dont-check production fires), no reward will be given. 

- motivation paramter = 1.5, it means that when model's mental clock exceeds motivation clock (> 1.5s), it would stop checking the rule anymore, but if the mental clock is within the motivation clock(< 1.5s), it would continue retrieving and checking the rule.

- If check-pass fires, it will deliver a reward (equal to the total duration time the model spends on retrieval).


In [2]:
from simon_device import *
import actr
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

## Model data (interactive)

This interactive figure is generated through adjusting motivation parameter

In [7]:
@interact
def model_output(motivation=(0.1, 3, 0.5), model_name=['model3','model2','model1']):
    task = run_experiment(model=f'simon-motivation-{model_name}', visible=False, trace=False, param_set={'motivation':motivation})
    df = task.process_trace_data()
    fig, axes = plt.subplots(1, 2, figsize=(15, 4))
    sns.pointplot(x="condition_stimulus", y="response_time", hue="condition_cue", dodge=True, height=4, aspect=0.7, palette="Set2", markers=["x", "o"],
                  data=df, ax=axes[0]).set(title=f'Motivation={motivation}')
    sns.pointplot(x="condition_stimulus", y="accuracy", hue="condition_cue", dodge=True, height=4, aspect=.7, palette="Set2", markers=["x", "o"],
                  data=df, ax=axes[1]).set(title=f'Motivation={motivation}')

interactive(children=(FloatSlider(value=1.1, description='motivation', max=3.0, min=0.1, step=0.5), Dropdown(d…

In [4]:
@interact
def utility_output(motivation=(0.1, 3, 0.5), model_name=['model3','model2','model1']):
    task = run_experiment(model=f'simon-motivation-{model_name}', visible=False, trace=False, param_set={'motivation':motivation})
    df = task.process_trace_data()
    #fig, axes = plt.subplots(2, 1, figsize=(15, 10))
    g = sns.FacetGrid(df, row="condition_stimulus", col="condition_cue", hue="production", height=4, aspect=1, margin_titles=True)
    g.map(sns.lineplot, "time", ":u")
    g.add_legend()

interactive(children=(FloatSlider(value=1.0, description='motivation', max=3.0), Dropdown(description='model_n…